## IMPORTING THE REQUIRED LIBRARIES

In [1]:
import pandas as pd ; import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.models import Sequential

## DISABLING THE SETTING WITH COPY WARNING

In [2]:
pd.options.mode.chained_assignment = None

## IMPORTING DATA

In [3]:
def importData(path: str) -> pd.DataFrame:
    return pd.read_csv(path)

faces = importData('../preprocessing/data/facesDataClean.csv')

## ENCODING THE GENDER COLUMN INTO ZEROS AND ONES

In [4]:
def encodingTarget(dataframe: pd.DataFrame, target: str) -> (pd.DataFrame, LabelEncoder):
    
    encoder = LabelEncoder().fit(dataframe[target])
    dataframe[target] = encoder.transform(dataframe[target])

    return dataframe, encoder

faces, genderEncoder = encodingTarget(faces, 'gender')

## GETTING ONLY THE DATA OF THE PEOPLE THAT IS FROM THE WHITE ETHNICITY

In [5]:
facesWhite = faces[faces['ethnicity'] == 'White']

## PREPARING THE PIXELS COLUMN FOR THE NEURAL NETWORK

In [6]:
def formatPixels(pixels: str) -> np.array:

    pixelsArray = np.array(pixels.split(), 'float64')
    pixelsArrayReshaped = np.reshape(pixelsArray, (48, 48))
    pixelsNormalized = pixelsArrayReshaped / 255.0

    return pixelsNormalized

facesWhite['pixels'] = facesWhite['pixels'].apply(formatPixels)

## PERFORMING THE X-Y TRAIN-TEST SPLIT

In [7]:
def split(dataframe, pixels, gender):

    X = dataframe[pixels]
    y = dataframe[gender]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split(facesWhite, 'pixels', 'gender')

## PERFORMING THE FINAL RESHAPE IN THE TRAINING AND THE TESTING DATA

In [8]:
X_train = np.reshape(X_train.to_list(), (len(X_train), 48, 48, 1)) / 255.0
X_test = np.reshape(X_test.to_list(), (len(X_test), 48, 48, 1)) / 255.0

## DEFINING AND COMPILING THE STRUCTURE OF THE NEURAL NETWORK

In [9]:
def neuralNetwork():

    model = Sequential()
    model.add(Conv2D(256, (3, 3), activation = 'relu', padding = 'same', input_shape = (48, 48, 1)))
    model.add(MaxPool2D(2, 2))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation = 'relu', padding = 'same'))
    model.add(MaxPool2D(2, 2))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), activation = 'relu', padding = 'same'))
    model.add(MaxPool2D(2, 2))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(optimizer = SGD(0.01), loss = 'binary_crossentropy', metrics = ['accuracy'])

    return model

genderClassifier = neuralNetwork()

## DEFINING THE VALUES FOR THE EARLY STOPPING AND THE LR AIMING TO REDUCE THE OVERFITTING

In [10]:
earlyStop = EarlyStopping(monitor = 'val_accuracy', patience = 3)
reduceLR = ReduceLROnPlateau(monitor = 'val_accuracy', patience = 1, min_lr = 0.000001)

## TRAINING THE NEURAL NETWORK AND DISPLAYING THE PROCESS

In [11]:
genderClassifierFit = genderClassifier.fit(X_train, y_train, validation_data = (X_test, y_test),
                                           epochs = 50, callbacks = [reduceLR, earlyStop], verbose = 0)

Epoch 1/50
252/252 [==============================] - 49s 195ms/step - loss: 0.5130 - accuracy: 0.7425 - val_loss: 0.6907 - val_accuracy: 0.5367
Epoch 2/50
252/252 [==============================] - 49s 193ms/step - loss: 0.3764 - accuracy: 0.8316 - val_loss: 0.8206 - val_accuracy: 0.5367
Epoch 3/50
252/252 [==============================] - 49s 195ms/step - loss: 0.3007 - accuracy: 0.8776 - val_loss: 0.5439 - val_accuracy: 0.7103
Epoch 4/50
252/252 [==============================] - 49s 194ms/step - loss: 0.2896 - accuracy: 0.8819 - val_loss: 0.3496 - val_accuracy: 0.8393
Epoch 5/50
252/252 [==============================] - 49s 195ms/step - loss: 0.2813 - accuracy: 0.8853 - val_loss: 0.3503 - val_accuracy: 0.8423
Epoch 6/50
252/252 [==============================] - 49s 196ms/step - loss: 0.2747 - accuracy: 0.8874 - val_loss: 0.3369 - val_accuracy: 0.8487
Epoch 7/50
252/252 [==============================] - 49s 195ms/step - loss: 0.2684 - accuracy: 0.8913 - val_loss: 0.3324 - val_ac